# Day 25: one final set of cellular automata

* <https://adventofcode.com/2021/day/25>

As always, day 25 is a simpler, one-star problem. It's another cellular automata challenge.

Like most such problems before, I rely on [`scipy.signal.convolve2d()` function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.convolve2d.html#scipy.signal.convolve2d) to do the work. To 'move' seacucumbers, make two sets of changes, one for each 'herd'. Occupied cells become empty if there is an adjacent cell to the the right or below it to move to, and empty cells become occupied if there is an adjacent cell in the opposite direction that is occupied. For each of these moves, we need a kernel with the one cell in the direction we want to look, set to 1, the rest is set to 0.

We do need to then define the transition rules for the automata:

- a cell becomes an empty cell if 
  - it currently is an east-moving seacucumber and there is an empty cell to the east of it _and_ it doesn't have a south-moving seacucumber to the north.
  - it is a south-moving seacucumber and it has an empty cell to the south _and_ it has no east-moving seacucumber to the south-west.
  - it is a south-moving seacucumber and it has an east-moving seacucumber to the south _and_ it has an empty cell to the south-east.
- a cell becomes an east-moving seacucumber if
  - it currently is an empty cell and it has an east-moving seacucumber to its west 
- a cell becomes a south-moving seacucumber if
  - it currently is an empty cell and it has a south-moving seacucumber to its north _and_ no east-moving seacucumber to its west
  - it currently is an east-moving seacucumber and there is an empty cell to its east _and_ a south-moving seacucumber to its north.

To fullfill those rules, we'll need 6 different convolve kernels to count neighbours in all 4 straight directions, plus two for the south-west and south-east diagonals, then use those to count the 3 different types of cell:

- the east, south, and south-east kernels to count empty cells
- the west, south and south-west kernels to count east-moving seacucumbers
- the north kernel to count south-moving seacucumbers

In [1]:
from __future__ import annotations

from enum import Enum
from itertools import count
from typing import TYPE_CHECKING, Final

import numpy as np
from scipy.signal import convolve2d


class Seafloor(Enum):
    empty = 0, "."
    east = 1, ">"
    south = 2, "v"

    if TYPE_CHECKING:
        value: str
        int: int

    def __new__(cls, int: int, value: str):
        instance = object.__new__(cls)
        instance._value_ = value
        instance.int = int
        return instance


class Kernel(Enum):
    # index into 3x3 array for each direction; 0 is the bottom-right-hand corner
    # and 8 is the top left.
    se = 0
    s = 1
    sw = 2
    e = 3
    w = 5
    n = 7

    def __init__(self, idx: int):
        array = np.zeros((9,))
        array[idx] = True
        self.array = array.reshape((3, 3))


DIRECTIONS: Final[dict[Seafloor, tuple[Kernel, ...]]] = {
    Seafloor.empty: (Kernel.e, Kernel.s, Kernel.se),
    Seafloor.east: (Kernel.w, Kernel.s, Kernel.sw),
    Seafloor.south: (Kernel.n,),
}


class SeacucumberSim:
    def __init__(self, map: str) -> None:
        cells = [[Seafloor(c).int for c in ln] for ln in map.splitlines()]
        self._matrix = np.array(cells)

    def __str__(self) -> str:
        m = {a.int: a.value for a in Seafloor}
        return "\n".join(["".join(map(m.get, row)) for row in self._matrix])

    def run_simulation(self) -> int:
        m = self._matrix
        full = {s: np.full(m.shape, s.int) for s in Seafloor}
        prev = np.zeros(m.shape)
        for step in count(1):
            # counts for each direction for specific seafloor cells
            adjacent: dict[tuple[Kernel, Seafloor], np.array] = {
                (k, s): convolve2d(m == s.int, k.array, mode="same", boundary="wrap")
                > 0
                for s, ks in DIRECTIONS.items()
                for k in ks
            }
            rules = {
                Seafloor.empty: (
                    (
                        (m == Seafloor.east.int)
                        & adjacent[Kernel.e, Seafloor.empty]
                        & ~adjacent[Kernel.n, Seafloor.south]
                    )
                    | (
                        (m == Seafloor.south.int)
                        & adjacent[Kernel.s, Seafloor.empty]
                        & ~adjacent[Kernel.sw, Seafloor.east]
                    )
                    | (
                        (m == Seafloor.south.int)
                        & adjacent[Kernel.s, Seafloor.east]
                        & adjacent[Kernel.se, Seafloor.empty]
                    )
                ),
                Seafloor.east: (
                    (m == Seafloor.empty.int) & adjacent[Kernel.w, Seafloor.east]
                ),
                Seafloor.south: (
                    (
                        (m == Seafloor.empty.int)
                        & adjacent[Kernel.n, Seafloor.south]
                        & ~adjacent[Kernel.w, Seafloor.east]
                    )
                    | (
                        (m == Seafloor.east.int)
                        & adjacent[Kernel.n, Seafloor.south]
                        & adjacent[Kernel.e, Seafloor.empty]
                    )
                ),
            }
            m = np.select(list(rules.values()), [full[s] for s in rules], default=m)
            if np.array_equal(m, prev):
                return step
            prev = m


test_map = SeacucumberSim(
    """\
v...>>.vv>
.vv>>.vv..
>>.>v>...v
>>v>>.>.v.
v>v.vv.v..
>.>>..v...
.vv..>.>v.
v.v..>>v.v
....v..v.>
"""
)
assert test_map.run_simulation() == 58


In [3]:
import aocd

seafloor = SeacucumberSim(aocd.get_data(day=25, year=2021))
print("Part 1:", seafloor.run_simulation())

Part 1: 351
